In [2]:
from plyfile import PlyData, PlyElement
from glob import glob
from utils.pointcloud import grid_subsample_simple
import os
from omegaconf import OmegaConf
import shutil
import pandas as pd
from scipy.spatial import KDTree
import numpy as np


cfg = OmegaConf.load("./config/subsampling.yaml")
out_dir = cfg.data.preprocess_raw.output_dir_subsampling


for proj_fp in glob(os.path.join(cfg.data.preprocess_raw.raw_dir, "*.FwfProj")):
    project_name = os.path.basename(proj_fp)
    print(f"{project_name}")

    pcd_fp = list(glob(os.path.join(proj_fp,'labeled',"*_pointcloud.ply")))[0]
    waveform_fp = list(glob(os.path.join(proj_fp,'labeled',"*_waveform.npy")))[0]
    meta_fp = waveform = list(glob(os.path.join(proj_fp,'labeled',"*_metadata.json")))[0]

    pcd = pd.DataFrame.from_dict(PlyData.read(pcd_fp).elements[0].data)
    xyz = pcd[['x','y','z']].to_numpy()
    kd_tree = KDTree(xyz)

    # get sub inds
    sub = grid_subsample_simple(xyz, cfg.data.preprocess_raw.voxel_size)
    _,sub_ids = kd_tree.query(sub['points'])

    # subsample point cloud
    sub_pcd = pcd.loc[sub_ids]

    # subsample waveform
    wfm = np.load(waveform_fp)
    wfm_sub = wfm[sub_ids]

    proj_out_dir = os.path.join(out_dir,project_name,'labeled')
    os.makedirs(proj_out_dir, exist_ok=True)
    PlyData([PlyElement.describe(sub_pcd.to_records(index=False),'vertex')]).write(os.path.join(proj_out_dir, os.path.basename(pcd_fp)))
    np.save(os.path.join(proj_out_dir, os.path.basename(waveform_fp)),wfm_sub)
    shutil.copy(meta_fp,os.path.join(proj_out_dir,os.path.basename(meta_fp)))

    assert sub_pcd.shape[0] == wfm_sub.shape[0], "Something went wrong"
shutil.copy(os.path.join(cfg.data.preprocess_raw.raw_dir, 'class_dict.json'), os.path.join(out_dir,'class_dict.json'))



2023-08-28_FW_EingangBauing.FwfProj
2024-03-22_FW_Koenigshuegel.FwfProj
2024-04-05_FW_Westbahnhof_01.FwfProj
2024-04-05_FW_Westbahnhof_02.FwfProj
2024-04-05_FW_Westbahnhof_03.FwfProj
2024-04-05_FW_Westbahnhof_04.FwfProj
2024-04-05_FW_Westbahnhof_05.FwfProj
2024-05-10_FW_RWTH_Zentrum_01.FwfProj
2024-07-31_FW_Bruecke_Koenigstr.FwfProj
2024-07-31_FW_Bruecke_Turmstr.FwfProj
2024-08-02_FW_Bruecke_A44_VerlautenheidenerStr.FwfProj
2024-08-02_FW_Bruecke_Deltourserb.FwfProj
2024-08-02_FW_Bruecke_Kasinostrasse.FwfProj
2024-08-02_FW_Bruecke_RotheErde.FwfProj
2024-08-02_FW_Bruecke_Rottstrasse.FwfProj


'D:\\Projekte\\GIA_220412_PCS\\02_Datasets\\FullWaveForm\\full_waveform_software\\FullWaveformParse_mass\\output\\FWF_Aachen_labeled\\/../FWF_Subsampled/0.01/class_dict.json'